In [11]:
from helpers import *
from keras.layers import *
from keras.models import Model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
import keras
weight_decay       = 1e-4

In [2]:
x_train, _, y_train= load_training_data()
x_test, _, y_test = load_test_data()
input_shape = list(x_train.shape[1:])
classes = y_train.shape[1]

Loading data: /home/alex/CS230/CNN/data_batch_1
Loading data: /home/alex/CS230/CNN/data_batch_2
Loading data: /home/alex/CS230/CNN/data_batch_3
Loading data: /home/alex/CS230/CNN/data_batch_4
Loading data: /home/alex/CS230/CNN/data_batch_5
Loading data: /home/alex/CS230/CNN/test_batch


In [15]:
def buildingBlock(X,filter_size,filters,stride=1):
    
    X_shortcut = X
    
    #convolution block,else identity block
    if(stride>1):
        X_shortcut = Conv2D(filters,kernel_size=(1,1),strides=stride,padding='same')(X_shortcut)
        X_shortcut = BatchNormalization(axis=3)(X_shortcut)
    #first layer
    
    X = Conv2D(filters,kernel_size=filter_size,strides=stride,padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    #second layer
    
    X = Conv2D(filters,kernel_size=filter_size,strides=(1,1),padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = add([X,X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [16]:
def model(input_shape,classes,name):
    
    X_input = Input(input_shape)
    
    #Stage 1
    #input 32x32x3 output 32x32x16
    X = Conv2D(filters=16,kernel_size=3,strides=(1,1),padding='same',kernel_regularizer=keras.regularizers.l2(weight_decay),
              kernel_initializer="he_normal")(X_input)

    for i in range(5):
        X = buildingBlock(X,filter_size=3,filters=16,stride=1)
    
    X = buildingBlock(X,filter_size=3,filters=16,stride=2)
    for i in range(4):
        X = buildingBlock(X,filter_size=3,filters=16,stride=1)
        
    
    X = buildingBlock(X,filter_size=3,filters=16,stride=2)
    for i in range(4):
        X = buildingBlock(X,filter_size=3,filters=16,stride=1)
        
    X = BatchNormalization(momentum=0.9, epsilon=1e-5)(X)
    X = Activation('relu')(X)
    X = GlobalAveragePooling2D()(X)
    
    #input64 output 10
    X = Dense(classes,activation='softmax',kernel_initializer="he_normal", kernel_regularizer=regularizers.l2(weight_decay))(X)

    return Model(inputs = X_input,outputs=X,name=name)
    

In [17]:
optimizer = optimizers.Adam(lr=0.001,beta_1=0.9,beta_2=0.999,epsilon=1e-8,decay=0.0)
ResNet32 = model(input_shape=input_shape,classes = classes,name='ResNet32')
ResNet32.compile(optimizer=optimizer,loss='categorical_crossentropy', metrics = ['accuracy'])


In [ ]:
datagen = ImageDataGenerator(
        rotation_range=30,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True)  # randomly flip images


# Train model
results = ResNet32.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size = 250),
                                 epochs = 1,
                                 steps_per_epoch=200,  # data_size/batch size
                                 validation_data=(x_test, y_test))

# Plot train / validation results
plot_results(results)

# Print model architecture
ResNet32.summary()


Epoch 1/1
199/200 [============================>.] - ETA: 1s - loss: 0.9645 - acc: 0.6579

In [ ]:
#more specfic version

#https://blog.csdn.net/bryant_meng/article/details/81187434